## Nss Midcourse project: Creating pathfinder 2e monster encounter
### Notebook to clean and reformat .csv files for next processign steps 
csv files downloaded from https://pf2.tools/

In [137]:
#Import libraries 
import pandas as pd
import numpy as np
import requests
import matplotlib as plt


import requests
from bs4 import BeautifulSoup
import pandas as pd
from io import StringIO
import re

### clean and save class base statistic dataframe 

In [44]:
class_stats= pd.read_csv('../input_data/all_class_stats and attributes.csv')
class_stats= class_stats.dropna()

#select just the columns with statistic information 
class_stats= class_stats[['Class', 'Primary Ability', 'Hit Points per Level','Perception', 'Fortitude', 'Reflex', 'Will', 'Skills',
       'Defenses', 'Attacks', 'Spells', 'Class/Spell DC']]

#Clean up the columns to be list for cleaner organizing later 
class_stats['Attacks'] = class_stats['Attacks'].to_list()
class_stats['Defenses'] = class_stats['Defenses'].to_list()
class_stats['Hit Points per Level']= class_stats['Hit Points per Level'].astype(int)
class_stats['Skills'] = [x.split('\n') for x in class_stats['Skills']]

#Split out new columns for multiple abilities (i.e. new row for 'dexterity' build vs 'strength' build
class_stats['Primary Ability']= [x.split('or') for x in class_stats['Primary Ability']]
#class_stats['Primary Ability']= class_stats.explode('Primary Ability')

#save dataset as a clean .csv in 'output_data' file 
class_stats.to_csv('../output_data/all_class_base_stats.csv')

#view final dataframe 
class_stats

,Class,Primary Ability,Hit Points per Level,Perception,Fortitude,Reflex,Will,Skills,Defenses,Attacks,Spells,Class/Spell DC
0,Alchemist,[Intelligence],8,Trained,Expert,Expert,Trained,"[Crafting, +3 of Choice]","Light, Unarmored","Simple, Unarmed, Alchemical Bombs",--,Trained
1,Barbarian,[Strength],12,Expert,Expert,Trained,Expert,"[Athletics, +3 of Choice]","Light, Medium, Unarmored","Simple, Martial, Unarmed",--,Trained
2,Bard,[Charisma],8,Expert,Trained,Trained,Expert,"[Occultism and Performance, + 4 of Choice]","Light, Unarmored","Simple, Unarmed, Longsword, Rapier, Sap, Short...",Occult,Trained
3,Champion,"[Strength , Dexterity]",10,Trained,Expert,Trained,Expert,"[Religion and 1 of your Deity's, +2 of Choice]","All, Unarmored","Simple, Martial, Unarmed",Divine,Trained
4,Cleric,[Wisdom],8,Trained,Trained,Trained,Expert,"[Religion and 1 of your Deity's, +2 of Choice]","Unarmored, Armor noted by Doctrine","Simple, Deity's Favored Weapon, Unarmed",Divine,Trained
5,Druid,[Wisdom],8,Trained,Trained,Trained,Expert,"[Nature and 1 from your Order, +2 of Choice]","Light, Medium, Unarmored","Simple, Unarmed",Primal,Trained
6,Fighter,"[Strength , Dexterity]",10,Expert,Expert,Expert,Trained,"[Acrobatics OR Athletics, +3 of Choice]","All, Unarmored","Advanced; EXPERT: Simple, Martial, Unarmed",--,Trained
7,Monk,"[Strength , Dexterity]",10,Trained,Expert,Expert,Expert,[4 of Choice],EXPERT: Unarmored,"Simple, Unarmed",--,Trained
8,Ranger,"[Strength , Dexterity]",10,Expert,Expert,Expert,Trained,"[Nature and Survival, +4 of Choice]","Light, Medium, Unarmored","Simple, Martial, Unarmed",--,Trained
9,Rogue,"[Dexterity , Other]",8,Expert,Trained,Expert,Expert,"[Stealth and 1 from your Racket, +7 of Choice]","Light, Unarmored","Simple, Unarmed, Rapier, Sap, Shortbow, Shorts...",--,Trained


### clean and save: Equipment datasets

In [76]:
weapon_list= pd.read_csv('../input_data/weapon_equipment_list.csv')
weapon_list= weapon_list.dropna()

#replace empty row symbol with values 
weapon_list['Level']= weapon_list['Level'].replace('—', 0)
weapon_list['Price']= weapon_list['Price'].replace('—', 0)
weapon_list['Bulk']= weapon_list['Bulk'].replace('—', 0)
weapon_list

#clean up strings in columns
char_to_remove= r'*'
weapon_list['Group']= [x.replace(char_to_remove, '') for x in weapon_list['Group']]
weapon_list['Weapon Traits']= [x.replace(char_to_remove, '') for x in weapon_list['Weapon Traits']]
weapon_list['Weapon Traits']= weapon_list['Weapon Traits'].to_list()

#divide more categories 
weapon_list['Damage_type']= [x.split(' ')[-1] for x in weapon_list['Damage']]
weapon_list['Damage']= [x.split(' ')[0] for x in weapon_list['Damage']]

#save dataframe 
weapon_list.to_csv('../output_data/weapon_list_stats.csv')

weapon_list

,Name,Category,Level,Price,Damage,Bulk,Hands,Group,Weapon Traits,Damage_type
0,Fist,Unarmed,0,0,1d4,0,1,Brawling,"Agile, Finesse, Nonlethal, Unarmed",B
1,Clan Dagger,Simple,0,2 gp,1d4,L,1,Knife,"Agile, Dwarf, Parry, Uncommon, Versatile B",P
2,Club,Simple,0,0,1d6,1,1,Club,Thrown 10 ft.,B
3,Dagger,Simple,0,2 sp,1d4,L,1,Knife,"Agile, Finesse, Thrown 10 ft., Versatile S",P
4,Gauntlet,Simple,0,2 sp,1d4,L,1,Brawling,"Agile, Free-Hand",B
...,...,...,...,...,...,...,...,...,...,...
99,Fire Poi,Advanced,0,5 gp,1d4,L,1,Flail,"Agile, Backswing, Finesse, Twin, Uncommon",F
100,Gnome Flickmace,Advanced,0,3 gp,1d8,2,1,Flail,"Gnome, Reach, Uncommon",B
101,Orc Necksplitter,Advanced,0,2 gp,1d8,1,1,Axe,"Forceful, Orc, Sweep, Uncommon",S
102,Rhoka Sword,Advanced,0,4 gp,1d8,2,1,Sword,"Deadly d8, Two-Hand 1d10, Uncommon",S


In [102]:
armor_list= pd.read_csv('../input_data/armor_equipment_list.csv')
armor_list= armor_list.dropna()
armor_list.columns= armor_list.loc[0]
armor_list= armor_list.drop(0, axis=0)

#clean up text 
armor_list['Armor Traits']= [x.replace(char_to_remove, '') for x in armor_list['Armor Traits']]
armor_list= armor_list.replace('—', 0)
armor_list['Armor Traits']= armor_list['Armor Traits'].replace(0, 'none')

#take out only the number values for rows
armor_list['AC Bonus']= armor_list['AC Bonus'].str.extract('(\\d+)').fillna(0)
armor_list['Dex Cap']= armor_list['Dex Cap'].str.extract('(\\d+)').fillna(0)
armor_list['Check Penalty']= armor_list['Check Penalty'].str.extract('(\\d+)').fillna(0)
armor_list['Speed Penalty']= armor_list['Speed Penalty'].str.extract('(\\d+)').fillna(0)

#remove columns not important for later analysis 
armor_list= armor_list.drop(['Group', 'Price'], axis=1)

#save to output file
armor_list.to_csv('../output_data/armor_list_stats.csv')

armor_list

,Name,Category,Level,AC Bonus,Dex Cap,Check Penalty,Speed Penalty,Strength,Bulk,Armor Traits
2,Explorer's Clothing,Unarmored,0,0,5,0,0,0,L,Comfort
3,Padded Armor,Light,0,1,3,0,0,10,L,Comfort
4,Leather Armor,Light,0,1,4,1,0,10,1,none
5,Studded Leather Armor,Light,0,2,3,1,0,12,1,none
6,Chain Shirt,Light,0,2,3,1,0,12,1,"Flexible, Noisy"
7,Hide Armor,Medium,0,3,2,2,5,14,2,none
8,Scale Mail,Medium,0,3,2,2,5,14,2,none
9,Chain Mail,Medium,0,4,1,2,5,16,2,"Flexible, Noisy"
10,Breastplate,Medium,0,4,1,2,5,16,2,none
11,Splint Mail,Heavy,1,5,1,3,10,16,3,none


### clean and save spell list

In [185]:
spell_list= pd.read_csv('../input_data/spell_list.csv')
spell_list= spell_list.dropna(subset=['name'])
spell_list= spell_list.dropna(axis=1, how='all')

#drop columns that I do not want 
columns_to_drop= ['descriptor', 'components', 'costly_components', 'dismissible', 'shapeable', 'description',
       'description_formatted', 'source', 'full_text', 'verbal', 'somatic',
       'material', 'focus', 'divine_focus', 
       'SLA_Level', 'domain', 'short_description', 'linktext', 'id',
       'material_costs', 'mythic_text', 'augmented',
       'mythic', 'bloodrager', 'shaman', 'psychic', 'medium', 'mesmerist',
       'occultist', 'spiritualist', 'summoner unchained', 'linktext.1',
       'custom linktext']

spell_list= spell_list.drop(columns_to_drop, axis=1)

#view dataframe 
spell_list

,name,school,subschool,spell_level,casting_time,range,area,effect,targets,duration,...,lawful,light,mind_affecting,pain,poison,shadow,sonic,water,bloodline,patron
0,Acid Arrow,conjuration,creation,"sorcerer/wizard 2, magus 2, bloodrager 2",1 standard action,long (400 ft. + 40 ft./level),NaN,one arrow of acid,NaN,1 round + 1 round per three levels,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
1,Acid Fog,conjuration,creation,"sorcerer/wizard 6, magus 6, unchained summoner 6",1 standard action,medium (100 ft. + 10 ft./level),NaN,"fog spreads in 20-ft. radius, 20 ft. high",NaN,1 round/level,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
2,Acid Splash,conjuration,creation,"sorcerer/wizard 0, summoner/unchained summoner...",1 standard action,close (25 ft. + 5 ft./2 levels),NaN,one missile of acid,NaN,instantaneous,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
3,Aid,enchantment,compulsion,"cleric/oracle 2, inquisitor 2, alchemist 2, sh...",1 standard action,touch,NaN,NaN,living creature touched,1 min./level,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,Ancestors (4)
4,Air Walk,transmutation,NaN,"cleric/oracle 4, druid 4, alchemist 4, occulti...",1 standard action,touch,NaN,NaN,creature (Gargantuan or smaller) touched,10 min./level,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2901,"Minor Reversion, Mass",conjuration,healing,"cleric/oracle 6, psychic 5, sorcerer/wizard 6",30 minutes,close (25 ft. + 5 ft./2 levels),NaN,NaN,"one creature/level, no two of which can be mor...",24 hours,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
2902,Eroding Ray,transmutation,NaN,"cleric/oracle 2, druid 2, sorcerer/wizard 2",1 standard action,close (25 ft. + 5 ft./2 levels),NaN,ray,NaN,instantaneous,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
2903,Inhibiting Patterns,illusion,pattern,"bard 3, psychic 4, sorcerer/wizard 4",1 standard action,medium (100 ft. + 10 ft./level),NaN,"one cylinder (10-ft. radius, 40-ft. high)/3 le...",NaN,concentration +1 round/level (D),...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
2904,Scintillating Wall,illusion,pattern,"bard 4, psychic 5, sorcerer/wizard 5",1 standard action,close (25 ft. + 5 ft./2 levels),NaN,wall with an area of up to one 10-ft. square/l...,NaN,1 round/level (D),...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN


### create subcategory columns for casting time, since that will impact battle decisions

In [164]:
spell_list['casting_time'].unique()

#Identify spells that only have a single-round action cost, and mark that in a new column 
casting_action=[]
for index, value in enumerate(spell_list['casting_time']):
    check_string= str(value)
    if 'action' in check_string:
        #add just the action number value to the list
        casting_action.append(re.findall(r'\d+',value))
    else:
        casting_action.append(0)

#add it as a column to the data frame 
spell_list['casting_action_cost']= casting_action

#Identify spells that takes a number of rounds to cast, and mark that on a column 
casting_action_round=[]
for index, value in enumerate(spell_list['casting_time']):
    check_string= str(value)
    if 'round' in check_string or 'rounds' in check_string :
        #add just the action number value to the list
        casting_action_round.append(re.findall(r'\d+',value))
    else:
        casting_action_round.append(0)

#add it as a column to the data frame 
spell_list['casting_round_cost']= casting_action_round

#Identify spells that take MULTIPLE rounds (time) to cast, and mark that on a column 
casting_action_time=[]
for index, value in enumerate(spell_list['casting_time']):
    check_string= str(value)
    if 'round' in check_string or 'rounds' in check_string or 'action' in check_string:
        #add just the action number value to the list
        casting_action_time.append(0)
    else:
        casting_action_time.append(value)

#add it as a column to the data frame 
spell_list['casting_time_long']= casting_action_time

0                0
1                0
2                0
3                0
4                0
           ...    
2901    30 minutes
2902             0
2903             0
2904             0
2998           NaN
Name: casting_time_long, Length: 2906, dtype: object

### more cleaning needed, but will skip for time for now

In [186]:
#save to output file
spell_list.to_csv('../output_data/spell_list.csv')

### Clean and save monster stats

In [195]:
beastiary_stats= pd.read_csv('../input_data/bestiary_all_stats.csv')
beastiary_stats= beastiary_stats.dropna(subset=['Name'])
beastiary_stats= beastiary_stats.dropna(axis=1, how='all')

#save to output 
#save to output file
beastiary_stats.to_csv('../output_data/bestiary_stats.csv')